# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression 
from pymystem3 import Mystem
import nltk
from nltk.stem import WordNetLemmatizer 
import re 
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics import f1_score
from nltk.corpus import stopwords
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [2]:
toxic_comments = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
toxic_comments.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
toxic_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


При первичном осмотре датасета нет пропусков, нет дубликатов, присутствует лишний столбец, который можно удалить

In [5]:
toxic_comments = toxic_comments.drop(['Unnamed: 0'], axis=1)
#toxic_comments = toxic_comments.loc[:10000]

In [6]:
target = toxic_comments['toxic']

In [7]:
toxic_comments.head(10)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


При более подробном рассмотрении отмечается наличие большого количества не нужных символов, которые необходимо удалить, или просто удалим все, кроме букв.

In [8]:
def clear_text(text):
    a = re.sub(r'[^a-zA-Z ]', ' ', text) 
    b = " ".join(a.split())
    return b


In [9]:
for a in range(toxic_comments.shape[0]):
    toxic_comments.loc[a, 'text_cl'] = clear_text(toxic_comments.loc[a, 'text'])

In [10]:
for a in range(toxic_comments.shape[0]):
    toxic_comments.loc[a, 'text'] = toxic_comments.loc[a, 'text'].lower()

In [11]:
toxic_comments.head(10)

,text,toxic,text_cl
0,explanation\nwhy the edits made under my usern...,0,Explanation Why the edits made under my userna...
1,d'aww! he matches this background colour i'm s...,0,D aww He matches this background colour I m se...
2,"hey man, i'm really not trying to edit war. it...",0,Hey man I m really not trying to edit war It s...
3,"""\nmore\ni can't make any real suggestions on ...",0,More I can t make any real suggestions on impr...
4,"you, sir, are my hero. any chance you remember...",0,You sir are my hero Any chance you remember wh...
5,"""\n\ncongratulations from me as well, use the ...",0,Congratulations from me as well use the tools ...
6,cocksucker before you piss around on my work,1,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK
7,your vandalism to the matt shirvington article...,0,Your vandalism to the Matt Shirvington article...
8,sorry if the word 'nonsense' was offensive to ...,0,Sorry if the word nonsense was offensive to yo...
9,alignment on this subject and which are contra...,0,alignment on this subject and which are contra...


Тексты почистили, далее необходимо лемматизировать текст

In [12]:
lem = WordNetLemmatizer()

In [13]:
def lemmatiz(text):
    a = nltk.word_tokenize(text)
    b = ' '.join([lem.lemmatize(w) for w in a])
    return b

In [14]:
for a in range(toxic_comments.shape[0]):
    toxic_comments.loc[a, 'text_cl_lm'] = lemmatiz(toxic_comments.loc[a, 'text_cl'])

Разделим все данные на тестовую и обучающую выборки, кросс валидацию использовать из за массивного объема данных затратно по времени, используем валидационную выборку

In [15]:
corpus_cl_lem = toxic_comments['text_cl_lm']

In [16]:
train, val_test, target_train, target_val_test = train_test_split(
    corpus_cl_lem, target, test_size=0.4, random_state=12345)
valid, test, target_valid, target_test = train_test_split(
    val_test, target_val_test, test_size=0.5, random_state=12345)

Оценим частоту упоминания определенных слов величиной TFIDF с использованием стоп слов

In [17]:
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
train_idf = count_tf_idf.fit_transform(train)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
test_idf = count_tf_idf.transform(test)
train_idf

<95575x121110 sparse matrix of type '<class 'numpy.float64'>'
	with 2598819 stored elements in Compressed Sparse Row format>

In [19]:
valid_idf = count_tf_idf.transform(valid)
valid_idf

<31858x121110 sparse matrix of type '<class 'numpy.float64'>'
	with 836790 stored elements in Compressed Sparse Row format>

In [20]:
train_idf

<95575x121110 sparse matrix of type '<class 'numpy.float64'>'
	with 2598819 stored elements in Compressed Sparse Row format>

В ходе подготовки данных были проведены следующие манипуляции: 
1. Текст комментариев приведен к строчному виду
2. Удалены неинформативные символы
3. проведена лиммитизация текста
4. Оценена частота употребления каждого слова


## Обучение

Для обучения использовали следующие модели: дерево решений, случайный лес и логистическая регрессия, параметры подбирали по валидационной выборке

In [21]:
def F1(model, TR, TG):
    pr = model.predict(TR)
    a = f1_score(TG, pr)
    return a

In [22]:
for i in range(100,151,10):
    model = DecisionTreeClassifier(random_state=12345, max_depth = i)
    model.fit(train_idf, target_train)
    predictions_valid = model.predict(valid_idf)
    print("max_depth =", i, ": ", end='')
    print(f1_score(target_valid, predictions_valid))

max_depth = 100 : 0.7174389216512215
max_depth = 110 : 0.7208563304900485
max_depth = 120 : 0.7215842902313195
max_depth = 130 : 0.7232068908398213
max_depth = 140 : 0.7229472988600694
max_depth = 150 : 0.7200791295746786


In [23]:
#params_TREE = {'max_depth': [ 1, 5, 10]}
TREE = DecisionTreeClassifier(max_depth = 150)
#TREE = GridSearchCV(model_TREE, param_grid=params_TREE, scoring='f1', cv=3, verbose=True, n_jobs=-1)
TREE.fit(train_idf, target_train)
#TREE.best_params_


DecisionTreeClassifier(max_depth=150)

In [24]:
#best_est = 0
#best_depth = 0
#best_result = 0
#for est in range(100, 121, 10):
#    for depth in range (100, 121, 20):
#        model = RandomForestClassifier(random_state=12345,n_estimators=est, max_depth=depth)
#        model.fit(train_idf, target_train)
#        predictions_valid = model.predict(valid_idf)
#       result = f1_score(target_valid, predictions_valid)
#        if result > best_result:
#            best_model = model
#            best_result = result
#            best_est = est
#            best_depth = depth
#print("Метрики качества", best_result, "Количество деревьев:", best_est, "Максимальная глубина:", best_depth)

In [25]:
params_RFC = {'n_estimators': range(10, 150, 10)}
model_RFC = RandomForestClassifier()
#RFC = GridSearchCV(model_RFC, param_grid=params_RFC, scoring='f1', cv=3)
RFC = RandomForestClassifier(n_estimators = 90)
RFC.fit(train_idf, target_train)
#RFC.best_params_

RandomForestClassifier(n_estimators=90)

При обучении модели кросс валидацией били подобраны параметры на 10 000 строк датасета. Наибольшее значения метрики при колличестве деревьев равное 90. 

In [26]:
model_LR = LogisticRegression(solver='lbfgs', class_weight = 'balanced', C = 10) 
model_LR.fit(train_idf, target_train)

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=10, class_weight='balanced')

In [27]:
#def F1_cv(model, features, target): 
#    a = cross_val_score(model, features, target, cv=3, scoring='f1') 
#    final_score = abs(sum(a) / len(a))
#    return final_score

In [28]:
TREE_F1 = F1(TREE, valid_idf, target_valid)
#TREE_F1_cv = F1_cv(TREE, train_idf, target_train)
TREE_F1

0.7247857613711272

In [29]:
RFC_F1 = F1(RFC, valid_idf, target_valid)
#RFC_F1_cv = F1_cv(RFC, train_idf, target_train)
RFC_F1

0.7014415781487101

In [30]:
LR_F1 = F1(model_LR, valid_idf, target_valid)
#LR_F1_cv = F1_cv(model_LR, train_idf, target_train)
LR_F1

0.7617823479005998

In [31]:
summ = pd.DataFrame(
    {'F1': [TREE_F1, RFC_F1, LR_F1],
    #'F1_cv': [TREE_F1_cv, RFC_F1_cv, LR_F1_cv]
    },
    index = ['DecisionTreeClassifier', 'RandomForestClassifier', 'LogisticRegression'])
summ

,F1
DecisionTreeClassifier,0.724786
RandomForestClassifier,0.701442
LogisticRegression,0.761782


## Выводы

In [32]:
LR_F1_T = F1(model_LR, test_idf, target_test)

In [33]:
print("результат метрики F1 на тестовой выборке =", LR_F1_T)

результат метрики F1 на тестовой выборке = 0.7517854540154497


Результат работы выбранной модели на тренировочной выборке удавлетворительный (0,75) 

### общий вывод

В ходе подготовки данных были проведены следующие манипуляции:

Текст комментариев приведен к строчному виду
Удалены неинформативные символы
проведена лиммитизация текста
Оценена частота употребления каждого слова

Для обучения изучали следующие модели: дерево решений, случайный лес и логистическая регрессия, параметры подбирали по валидационной выборке и кросс валидацией

Лучший результат показала логистическая регрессия со следующими параметрами:
LogisticRegression(solver='lbfgs', class_weight = 'balanced', C = 10)